In [1]:
import pandas as pd
import numpy as np
from collections import Counter

from cycler import cycler

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [9]:
### Set up fonts
from matplotlib.font_manager import fontManager, FontProperties

path = "/Users/gabrielbenitez/Library/Fonts/Sora-VariableFont_wght.ttf"

fontManager.addfont(path)
prop = FontProperties(fname=path)

plt.rcParams['font.sans-serif'] = prop.get_name()

In [10]:
### Set up color palettes
sns.color_palette("blend:#184918,#F1EAE4", as_cmap = True)
sns.blend_palette(['#000000', '#184918', '#184918',  '#F1EAE4',], as_cmap = True)

black_col = '#000000'
bg_col = '#F4F1E7'

col_list = ['#ED7D3A', 
            '#314CB6', 
            '#2FBF71', 
            '#D991BA',
            '#EFCA08', 
            '#64113F',
            bg_col, 
            black_col] 
            
            

pal = sns.color_palette(col_list)
pal

[(0.9294117647058824, 0.49019607843137253, 0.22745098039215686),
 (0.19215686274509805, 0.2980392156862745, 0.7137254901960784),
 (0.1843137254901961, 0.7490196078431373, 0.44313725490196076),
 (0.8509803921568627, 0.5686274509803921, 0.7294117647058823),
 (0.9372549019607843, 0.792156862745098, 0.03137254901960784),
 (0.39215686274509803, 0.06666666666666667, 0.24705882352941178),
 (0.9568627450980393, 0.9450980392156862, 0.9058823529411765),
 (0.0, 0.0, 0.0)]

In [11]:
plt.rcParams['figure.facecolor']  = bg_col
plt.rcParams['figure.edgecolor']  = bg_col
plt.rcParams['axes.facecolor'] =  bg_col

#colour of text and ticks
mpl.rcParams['text.color'] = black_col
mpl.rcParams['axes.labelcolor'] = black_col
mpl.rcParams['axes.edgecolor'] = black_col
mpl.rcParams['xtick.color'] = black_col
mpl.rcParams['ytick.color'] = black_col

#top and right spines
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

#x ticks
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

plt.rcParams["font.family"] = prop.get_name()

In [51]:
df = pd.read_csv('travel.csv')
df = df[df['Type'] == 'Train'].reset_index(drop = True)

# subset to edi-ldn trips or ldn-edi
df = df[ ( (df['From'] == 'Edinburgh') & (df['To'] == 'London') ) | ( (df['From'] == 'London') & (df['To'] == 'Edinburgh') ) ].reset_index(drop = True)

df['Day'] = df['Day'].astype(int)
df['Year'] = df['Year'].astype(int)
df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']].astype(str).agg('-'.join, axis=1))
df['DotW'] = df['Date'].dt.day_name()

print(len(df))
Counter(df['Person'])

100


Counter({'Emma': 56, 'Gab': 44})

In [18]:
## Maybe All Train Journeys from London that have been taken?
## As a Circos Plot?
## And then also delays lol

44

In [ ]:
### Edinburgh to London in Numbers ###

In [ ]:
# N Trains have been taken between edinburgh and london 
# N% have been from London
# In Total, we have spent N hours on a train 
    # Of which N was because of delays
# Total Distance Travelled KM -> or over N laps of the meadows/hampstead heath
# Trains taken per month
# Trains taken per DotW
# Type of train taken (GAB specific)


In [30]:
### Train Journeys in the Past 3 Years, Cumulative (emma + gab)
    # Bar chart, with groups. 1 X for each year, then split between emma and gab. 

In [31]:
### Who has taken more journeys (live updating plot)

In [32]:
### LNER VS LUMO (gab)

In [ ]:
### Average time spent in location?